In [45]:
from src import Tool, PolicyDeepthEnvFixCoordinate_singlefile
name = 'ESPS_dataset/test_url'
data = dict(Tool.LoadObj(name))
total_url = 0
domain_count = 0
no_duplicate = []
for i, hrefs in enumerate(list(data['href'])):
    total_url +=len(hrefs)
    domain_count +=1
    for href in hrefs:
        if not href in no_duplicate:
            no_duplicate.append(href)
print('domain url',domain_count)
print('total url',total_url)
print('no_duplicate',len(no_duplicate))

domain url 92
total url 13934
no_duplicate 8721


In [46]:
import pandas as pd
traindf = pd.read_csv("./dataset_google_search/google_search_rank_train - contain news.csv")
validf = pd.read_csv("./dataset_google_search/google_search_rank_vali - contain_news.csv")
# testdf = pd.read_csv("./dataset_google_search/newdata_test_20 - remove error url.csv")
testdf = pd.read_csv("./dataset_google_search/newdata_test - remove error url.csv")
env = PolicyDeepthEnvFixCoordinate_singlefile.Env(testdf, negativeReward=-0.1, rewardweight=0.1)
positivepage,ans=env.GetPageANS(no_duplicate)
print('positivepage',len(positivepage))
print('ans',len(ans))
env.webQuit()

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST chromedriver version for 114.0.5735 google-chrome
[WDM] - Driver [C:\Users\Kuma\.wdm\drivers\chromedriver\win32\114.0.5735.90\chromedriver.exe] found in cache


positivepage 1272
ans 8721


In [47]:
no_duplicate_positive = []
print(len(positivepage))
for posurl in positivepage:
    if posurl not in no_duplicate_positive:
        no_duplicate_positive.append(posurl)
print(len(no_duplicate_positive))

1272
1236


In [48]:
no_duplicate_2 = [p[p.index(":"):] for p in no_duplicate]
no_duplicate_positive_2 = [p[p.index(":"):] for p in no_duplicate_positive] #只看http後的網址
print(len(no_duplicate_2),len(no_duplicate_positive_2))

8721 1236


In [49]:
negativepage = []
newpositive = []
for i,each_url in enumerate(no_duplicate_2):
    if each_url not in no_duplicate_positive_2:
        negativepage.append(no_duplicate[i])
    else:
        newpositive.append(no_duplicate[i])
print('negativepage len',len(negativepage))
print('positive len',len(newpositive))

negativepage len 8164
positive len 557


In [8]:
## train dataset down sample
from sklearn.utils import resample
# up_sample_positive = resample(no_duplicate_positive,n_samples=len(no_duplicate_positive)*2)
# down_sample_negative = resample(negativepage,n_samples=len(up_sample_positive)*2)
down_sample_negative = resample(negativepage,n_samples=len(no_duplicate_positive)*2)
# print(len(up_sample_positive))
print(len(down_sample_negative))

2582


In [50]:
# save positive dataset
import os
positive_label = [1]*len(no_duplicate_positive)
print(len(positive_label))
data = {'url':no_duplicate_positive,'label':positive_label }
df = pd.DataFrame(data)
path = 'ESPS_dataset/new'
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv('ESPS_dataset/new/test_positive.csv',index=False,encoding='utf_8_sig')

1236


In [30]:
# vali/test positive page all anchor(no contain all label data)
# import os
# positive_label = [1]*len(newpositive)
# print(len(positive_label))
# data = {'url':newpositive,'label':positive_label}
# df = pd.DataFrame(data)
# path = 'ESPS_dataset/new'
# if not os.path.exists(path):
#     os.makedirs(path)
# df.to_csv('ESPS_dataset/new/test_positive(no contain all label).csv',index=False,encoding='utf_8_sig')

542


In [51]:
# vali/test negative page no down sampling
negative_label = [0]*len(negativepage)
print(len(negative_label))
data = {'url':negativepage,'label':negative_label }
df = pd.DataFrame(data)
df.to_csv('ESPS_dataset/new/test_negative_pre.csv',index=False,encoding='utf_8_sig')

8164


In [ ]:
# negative page down sampling
negative_label = [0]*len(down_sample_negative)
print(len(negative_label))
data = {'url':down_sample_negative,'label':negative_label }
df = pd.DataFrame(data)
df.to_csv('ESPS_dataset/test92_negative_down.csv',index=False,encoding='utf_8_sig')

In [2]:
# vali/test concat positive and negative
import pandas as pd
df_pos = pd.read_csv('ESPS_dataset/new/test_positive.csv')
df_neg = pd.read_csv('ESPS_dataset/new/test_negative_pre.csv')
print(len(df_neg['url']))
print(len(df_pos['url']))
# total = pd.concat([df_pos,df_neg],ignore_index=True)
# df_shuffled = total.sample(frac=1, random_state=42)  # frac=1 表示将全部数据进行打乱
# df_shuffled.reset_index(drop=True, inplace=True)
# df_shuffled.to_csv('ESPS_dataset/new/test_no_down_pre2.csv', index=False,encoding='utf_8_sig')

8164
1236


In [10]:
# train concat positive and negative (1:2)
import pandas as pd
df_pos = pd.read_csv('ESPS_dataset/vali_positive(no contain all label).csv')
df_neg = pd.read_csv('ESPS_dataset/vali_negative.csv')

total = []
for i in range(len(df_pos)):
    pos_start = i+1
    pos_end = i+2
    neg_start = 2*i+1
    neg_end = 2*i+3
    total.append(pd.concat([df_pos.iloc[pos_start:pos_end],df_neg.iloc[neg_start:neg_end]],ignore_index=True))
total_df = pd.concat(total,ignore_index=True)
total_df.to_csv('ESPS_dataset/vali_no_down.csv',index=False,encoding='utf_8_sig')

In [23]:
# import pandas as pd
# testdf = pd.read_csv("./dataset_google_search/newdata_test - remove error url2.csv")
# testwebsiteInfo = testdf.groupby("Homepage")
# testlist = list(testwebsiteInfo.groups)[0:96]
# print('testlist len',len(testlist))
# new_test = []
# for url in testlist:
#     if url not in new_test:
#         new_test.append(url)
# print(len(new_test))

testlist len 95
95


In [42]:
# import pandas as pd
# vali = pd.read_csv("./ESPS_dataset/vali_positive_down.csv")
# vali_only_start_page = pd.read_csv("./ESPS_dataset/new/vali_positive(no contain all label).csv")
# collect_miss_vali = []
# only_start_page = []
# for url in vali_only_start_page['url']:
#     only_start_page.append(url)
# print(len(only_start_page))
# for url in vali['url']:
#     if url not in only_start_page:
#         collect_miss_vali.append(url)
# print(len(collect_miss_vali))

207
72
